# Collocates

> Functionality for collocation analysis.

In [ ]:
#| default_exp collocates

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import numpy as np
import time
import polars as pl
from fastcore.basics import patch
from scipy.stats import chi2

In [ ]:
#| export
from conc.corpus import Corpus
from conc.result import Result
from conc.core import logger, PAGE_SIZE, set_logger_state

In [ ]:
#| hide
import os

In [ ]:
#| hide
np.set_printoptions(suppress=True)
set_logger_state('verbose')

In [ ]:
#| hide
source_path = f'{os.environ.get("HOME")}/data/'
save_path = f'{os.environ.get("HOME")}/data/conc-test-corpora/'

path_to_toy_corpus = f'{save_path}toy.corpus'
path_to_brown_corpus = f'{save_path}brown.corpus'
path_to_reuters_corpus = f'{save_path}reuters.corpus'
path_to_gutenberg_corpus = f'{save_path}gutenberg.corpus'
path_to_gardenparty_corpus = f'{save_path}garden-party.corpus'
path_to_congress_corpus = f'{save_path}us-congressional-speeches-subset-100k.corpus'

In [ ]:
#| hide
reuters = Corpus().load(path_to_reuters_corpus)
brown = Corpus().load(path_to_brown_corpus)
gardenparty = Corpus().load(path_to_gardenparty_corpus)
gutenberg = Corpus().load(path_to_gutenberg_corpus)
congress = Corpus().load(path_to_congress_corpus)

2025-06-07 12:18:28 - INFO - memory_usage - init, memory usage: 973.8203125 MB


2025-06-07 12:18:28 - INFO - load - Load time: 0.769 seconds
2025-06-07 12:18:28 - INFO - memory_usage - init, memory usage: 987.546875 MB
2025-06-07 12:18:29 - INFO - load - Load time: 0.844 seconds
2025-06-07 12:18:29 - INFO - memory_usage - init, memory usage: 991.4765625 MB
2025-06-07 12:18:30 - INFO - load - Load time: 0.653 seconds
2025-06-07 12:18:30 - INFO - memory_usage - init, memory usage: 991.4765625 MB
2025-06-07 12:18:31 - INFO - load - Load time: 0.622 seconds
2025-06-07 12:18:31 - INFO - memory_usage - init, memory usage: 991.4765625 MB
2025-06-07 12:18:31 - INFO - load - Load time: 0.632 seconds


In [ ]:
#| export
class Collocates:
	""" Class for collocation analysis reporting. """
	def __init__(self,
			  corpus:Corpus # Corpus instance
			  ): 
		self.corpus = corpus


In [ ]:
#| exporti
@patch
def _shift_zeroes_to_end(self:Collocates,
						arr:np.ndarray # Numpy array of collocate frequencies to process
						):
	""" Move 0 value positions for punctuation and space removal """
	result = np.empty_like(arr)
	for col in range(arr.shape[1]):
		col_data = arr[:, col]
		mask = col_data != 0
		result[:mask.sum(), col] = col_data[mask]
		result[mask.sum():, col] = 0
	return result

In [ ]:
#| exporti
@patch
def _zero_after_value(self:Collocates,
					  arr:np.ndarray, # Numpy array of collocate frequencies to process
					  target: int # Target value to find in the array (e.g., an end-of-file token or a specific collocate frequency)
					  ):
	""" Set values from first occurence of target value to 0 in each column (for processing tokens outside text using eof token) """
	arr = arr.copy()  
	for col in range(arr.shape[1]):
		col_data = arr[:, col]
		idx = np.where(col_data == target)[0]
		if idx.size > 0:
			first_idx = idx[0]
			arr[first_idx:, col] = 0
	return arr

For learning journal = notebook pick up collocation dominoes

In [ ]:
#| exporti
@patch
def _get_tokens_in_context(self:Collocates,
							   token_positions:np.ndarray, # Numpy array of token positions in the corpus
							   index:str, # Index to use - lower_index, orth_index
							   context_length:int = 5, # Number of context words to consider on each side of the token
							   position_offset:int = 1, # offset to start retrieving context words - -1 for left, positive for right (may be adjusted by sequence_len)
							   exclude_punctuation:bool = True, # exclude punctuation from collocate retrieval
							   exclude_spaces:bool = True
							   ) -> Result:
	""" Get collocates in context for a given token index, operates one side at a time. """

	start_time = time.time()

	if context_length < 1:
		# return empty result
		return np.zeros((0, 0), dtype=np.int32)

	if position_offset < 0:
		position_offset_step = -1
	else:
		position_offset_step = 1
	
	tokens_for_removal = []
	if exclude_punctuation:
		tokens_for_removal += self.corpus.punct_tokens
	if exclude_spaces:
		tokens_for_removal += self.corpus.space_tokens
	len_tokens_for_removal = len(tokens_for_removal)

	collected = False
	context_tokens_arr = []
	while collected == False:
		new_positions = np.array(token_positions[0] + position_offset, dtype = token_positions[0].dtype)
		context_tokens_arr.append(self.corpus.get_tokens_by_index(index)[new_positions])
		position_offset += position_offset_step
		if len(context_tokens_arr) >= context_length: 
			context_tokens = np.array(context_tokens_arr, dtype = token_positions[0].dtype)
			if len_tokens_for_removal > 0: # cleaning spaces and punctuation and check if need more iterations
				context_tokens = np.where(np.isin(context_tokens, self.corpus.punct_tokens + self.corpus.space_tokens), 0, context_tokens)
			counts = np.count_nonzero(context_tokens, axis=0)
			if np.min(counts) < context_length:
				pass
			else:
				collected = True

	context_tokens = self._shift_zeroes_to_end(context_tokens)
	context_tokens = context_tokens[:context_length, :]

	if self.corpus.EOF_TOKEN in context_tokens:
		context_tokens = self._zero_after_value(context_tokens, self.corpus.EOF_TOKEN)

	logger.info(f"Collocates retrieved in {time.time() - start_time:.2f} seconds.")

	return context_tokens

In [ ]:
#| export
@patch
def collocates(self:Collocates, 
				token_str:str, # Token to search for
				effect_size_measure:str = 'logdice', # statistical measure to use for collocation calculation: logdice, mutual_information
				statistical_significance_measure:str = 'log_likelihood', # statistical significance measure to use, currently only 'log_likelihood' is supported
				order:str|None = None, # default of None orders by collocation measure, results can also be ordered by: collocate_frequency, frequency, log_likelihood
				order_descending:bool = True, # order is descending or ascending
				statistical_significance_cut: float|None = None, # statistical significance p-value to filter results, e.g. 0.05 or 0.01 or 0.001 - ignored if None or 0
				apply_bonferroni:bool = False, # apply Bonferroni correction to the statistical significance cut-off
				context_length:int|None=5, # Window size per side in tokens - use this for setting context lengths on left and right to same value
				context_left:int|None=None, # If context_left or context_right > 0 sets context lengths independently
				context_right:int|None=None, # see context_left
				min_collocate_frequency:int=5, # Minimum count of collocates
				page_size:int=PAGE_SIZE, # number of rows to return, if 0 returns all
				page_current:int=1, # current page, ignored if page_size is 0
				exclude_punctuation:bool=True, # exclude punctuation tokens
				exclude_spaces:bool=True # exclude space tokens
				) -> Result:
	""" Report collocates for a given token string. """

	if effect_size_measure not in ['logdice', 'mutual_information']:
		raise ValueError(f'Collocation measure must be one of "logdice" or "mutual_information".')
	
	if statistical_significance_measure not in ['log_likelihood']:
		raise ValueError(f'Statistical significance measure must be "log_likelihood".')

	if order not in [None, effect_size_measure, 'collocate_frequency', 'frequency', statistical_significance_measure]:
		raise ValueError(f'The order parameter must be None (default) or one of: {effect_size_measure}, collocate_frequency, frequency, {statistical_significance_measure}')

	token_sequence, index_id = self.corpus.tokenize(token_str, simple_indexing=True)

	index_column = 'lower_index'
	frequency_column = 'frequency_lower'
	columns = ['rank', 'token', 'collocate_frequency', 'frequency']

	start_time = time.time()
	debug = False

	sequence_len = len(token_sequence[0])
	token_positions = self.corpus.get_token_positions(token_sequence, index_id)

	if token_positions is None or token_positions[0].shape[0] == 0:
		logger.warning(f'Token "{token_str}" not found in the corpus.')
		return Result(type='collocates', df=pl.DataFrame(), title=f'No matches for "{token_str}"', description=f'{self.corpus.name}', summary_data={}, formatted_data=[])

	count_tokens = self.corpus.token_count
	tokens_descriptor = 'all tokens'
	total_descriptor = 'Total tokens'
	if exclude_punctuation and exclude_spaces:
		count_tokens = self.corpus.word_token_count
		tokens_descriptor = 'word tokens'
		total_descriptor = 'Total word tokens'
	elif exclude_punctuation:
		space_tokens_count = self.corpus.spaces.select(pl.len()).collect(engine='streaming').item()
		count_tokens = self.corpus.word_token_count + space_tokens_count
		tokens_descriptor = 'word and space tokens'
		total_descriptor = 'Total word and space tokens'
	elif exclude_spaces:
		punct_tokens_count = self.corpus.puncts.select(pl.len()).collect(engine='streaming').item()
		count_tokens = self.corpus.word_token_count + punct_tokens_count
		tokens_descriptor = 'word and punctuation tokens'
		total_descriptor = 'Total word and punctuation tokens'

	formatted_data = []
	formatted_data.append(f'Report based on {tokens_descriptor}')

	# if any of context_length, context_left, context_right are None - set them to 0
	if context_length is None:
		context_length = 0
	if context_left is None:
		context_left = 0
	if context_right is None:
		context_right = 0

	if context_left == 0 and context_right == 0:
		context_left = context_length
		context_right = context_length
	elif (context_left > 0 or context_right > 0) and context_length > 0:
		logger.warning('Context length is ignored if either context_left or context_right is set to a value greater than 0. To remove this warning, set context_length to None or 0.')

	formatted_data.append(f'Context tokens left: {context_left}, context tokens right: {context_right}')

	# getting context tokens
	left_tokens = self._get_tokens_in_context(token_positions=token_positions, index=index_column, context_length=context_left, position_offset=-1, exclude_punctuation=exclude_punctuation, exclude_spaces=exclude_spaces)
	right_tokens = self._get_tokens_in_context(token_positions=token_positions, index=index_column, context_length=context_right, position_offset=sequence_len, exclude_punctuation=exclude_punctuation, exclude_spaces=exclude_spaces)
	combined_tokens = np.concatenate([left_tokens.flatten(), right_tokens.flatten()])
	del left_tokens, right_tokens
	combined_tokens = combined_tokens[combined_tokens != 0] # removes punctuation and space placeholder
	# getting frequencies of collocates
	unique_token_ids, counts = np.unique(combined_tokens, return_counts=True)
	token_count_in_context_window = combined_tokens.shape[0]

	node_tokens = self._get_tokens_in_context(token_positions=token_positions, index=index_column, context_length=sequence_len, position_offset=0, exclude_punctuation=exclude_punctuation, exclude_spaces=exclude_spaces)
	unique_node_token_ids, node_counts = np.unique(node_tokens, return_counts=True)

	df = pl.DataFrame({
		'token_id': unique_token_ids,
		'collocate_frequency': counts
	})

	# for log liklihood calculation - need to have counts of node tokens to adjust collocate_frequency_outside_context
	node_frequency = np.zeros_like(unique_token_ids, dtype=np.int32)
	# use where to set node_frequency for node tokens
	for i, token_id in enumerate(unique_node_token_ids):
		node_frequency[unique_token_ids == token_id] = node_counts[i]
	df = df.with_columns(
		pl.lit(node_frequency).alias('node_frequency')
	)

	# adding frequency of collocates in corpus
	df = df.join(self.corpus.vocab.collect().select(['token_id', 'token', frequency_column]), on='token_id', how='left', maintain_order='left')
	df = df.rename({frequency_column: 'frequency'})

	filtering_descriptors = []
	if min_collocate_frequency > 1: # applying min_frequency filter
		df = df.filter(pl.col('collocate_frequency') >= min_collocate_frequency)
		filtering_descriptors.append(f'minimum collocation frequency ({min_collocate_frequency})')
	if len(filtering_descriptors) > 0:
		formatted_data.append(f'Filtered tokens by {(", ".join(filtering_descriptors))}')

	if effect_size_measure == 'logdice':
		# calculating collocation measure
		# from old code: logdice = 14 + math.log2((2 * collocate_count) / (node_frequency + loaded_corpora[corpus_name]['frequency_lookup'][collocate]))
		df = df.with_columns(
			(pl.lit(14) + ((2 * pl.col('collocate_frequency')) / (pl.lit(token_positions[0].shape[0]) + pl.col('frequency'))).log(2))
			.alias('logdice')
		)
		columns.append('logdice')
	elif effect_size_measure == 'mutual_information':
		# from old code: mi = math.log2((loaded_corpora[corpus_name]['token_count'] * collocate_count) / (node_frequency * loaded_corpora[corpus_name]['frequency_lookup'][collocate]))
		df = df.with_columns(
			(pl.lit(count_tokens) * pl.col('collocate_frequency') / (pl.lit(token_positions[0].shape[0]) * pl.col('frequency'))).log(2)
			.alias('mutual_information')
		)
		columns.append('mutual_information')

	if statistical_significance_measure == 'log_likelihood':
		# based on calculation for keyness: https://ucrel.lancs.ac.uk/llwizard.html
		# a = collocate frequency in context, is collocate_frequency
		# b = collocate frequency outside context ...
		# reminder: for individual tokens constituting the node, collocate_frequency_outside_context should exclude frequency of tokens in node
		df = df.with_columns(
			pl.max_horizontal([(pl.col('frequency') - pl.col('collocate_frequency') - pl.col('node_frequency')), pl.lit(0)]).alias('collocate_frequency_outside_context')
		)

		# c = total tokens in context windows, is token_count_in_context_window calculated above
		# d = total tokens outside context windows
		total_tokens_outside_context_window = count_tokens - token_count_in_context_window - (token_positions[0].shape[0] * sequence_len)

		# E1 = c*(a+b) / (c+d) 
		# E2 = d*(a+b) / (c+d)
		# E1 and E2
		df = df.with_columns(
			((pl.lit(token_count_in_context_window) * (pl.col('collocate_frequency') + pl.col('collocate_frequency_outside_context'))) / (pl.lit(token_count_in_context_window) + pl.lit(total_tokens_outside_context_window))).alias('expected_frequency1'),
			((pl.lit(total_tokens_outside_context_window) * (pl.col('collocate_frequency') + pl.col('collocate_frequency_outside_context'))) / (pl.lit(token_count_in_context_window) + pl.lit(total_tokens_outside_context_window))).alias('expected_frequency2'), 
		)

		# G2 = 2*((a*ln (a/E1)) + (b*ln (b/E2))) 
		# components of G2 as term1 and term 2 - (a*ln (a/E1)) (b*ln (b/E2))
		df = df.with_columns([
			pl.when(pl.col('collocate_frequency') > 0)
			.then(pl.col('collocate_frequency') * (pl.col('collocate_frequency') / pl.col('expected_frequency1')).log())
			.otherwise(0)
			.alias('term1'),
			pl.when(pl.col('collocate_frequency_outside_context') > 0)
			.then(pl.col('collocate_frequency_outside_context') * (pl.col('collocate_frequency_outside_context') / pl.col('expected_frequency2')).log())
			.otherwise(0)
			.alias('term2') # 0 if no reference frequency
		])

		# G2
		df = df.with_columns(
			(2 * (pl.col('term1') + pl.col('term2'))).alias('log_likelihood')
		)
		columns.append('log_likelihood')

	unique_collocates = df.select(pl.len()).item()

	if statistical_significance_cut is not None and statistical_significance_cut > 0:
		p = statistical_significance_cut
		# bonferroni correction
		if apply_bonferroni:
			p_value_descriptor = f'Keywords filtered based on p-value {p} with Bonferroni correction (based on {unique_collocates} tests)'
			p = p / unique_collocates # adjust by criteria
		else:
			p_value_descriptor = f'Keywords filtered based on p-value: {p}'
		cut = chi2.ppf(1 - p, df=1)		
		df = df.filter(pl.col('log_likelihood') > cut)
		formatted_data.append(p_value_descriptor)
		unique_collocates = df.select(pl.len()).item()

	formatted_data.append(f'Unique collocates: {unique_collocates:,.0f}')

	# prepare report information and paging ...
	if order is None:
		order = effect_size_measure
	df = df.sort(order, descending = order_descending).with_row_index('rank', offset=1)

	if page_size > 0:
		start = (page_current - 1) * page_size
		df = df.slice(start, page_size)

	if page_size != 0 and unique_collocates > page_size:
		formatted_data.append(f'Showing {page_size} rows')
		formatted_data.append(f'Page {page_current} of {unique_collocates // page_size + 1}')

	#formatted_data.append(f'{total_descriptor}: {count_tokens:,.0f}')
	if debug:
		columns = df.columns

	logger.info(f"Collocates calculated in {time.time() - start_time:.2f} seconds.")

	return Result(type = 'collocates', df = df.select(columns), title=f'Collocates of "{token_str}"', description=f'{self.corpus.name}', summary_data={}, formatted_data=formatted_data)
	

In [ ]:
collocates = Collocates(congress)

In [ ]:
for word in ["economy"]: # brown used 'i went in', 'any of us',  for testing "economy"
    %time collocates.collocates(word, order = None, order_descending = True, statistical_significance_cut = 0.0001, apply_bonferroni=True, effect_size_measure='logdice', context_length = 5, min_collocate_frequency = 5, page_current = 1).display()

2025-06-07 12:18:32 - INFO - _init_token_arrays - Created tokens_array in 0.081 seconds
2025-06-07 12:18:32 - INFO - _init_token_arrays - Created tokens_lookup in 0.086 seconds
2025-06-07 12:18:33 - INFO - _init_token_arrays - Created tokens_sort_order in 1.406 seconds
2025-06-07 12:18:33 - INFO - tokenize - Tokenization time: 1.59777 seconds
2025-06-07 12:18:34 - INFO - get_token_positions - Token indexing (3758) time: 0.55651 seconds
2025-06-07 12:18:34 - INFO - _get_tokens_in_context - Collocates retrieved in 0.10 seconds.
2025-06-07 12:18:34 - INFO - _get_tokens_in_context - Collocates retrieved in 0.11 seconds.
2025-06-07 12:18:34 - INFO - _get_tokens_in_context - Collocates retrieved in 0.07 seconds.
2025-06-07 12:18:34 - INFO - collocates - Collocates calculated in 0.95 seconds.


Collocates of "economy" 
 
 
 US Congressional Speeches Subset 100k 
 
 
 Rank 
 Token 
 Collocate Frequency 
 Frequency 
 Logdice 
 Log Likelihood 
 
 
 
 
 1 
 our 
 1,084 
 60,051 
 9.12 
 2,801.70 
 
 
 2 
 efficiency 
 60 
 732 
 8.77 
 329.93 
 
 
 3 
 stimulate 
 51 
 299 
 8.69 
 358.80 
 
 
 4 
 global 
 55 
 618 
 8.69 
 311.68 
 
 
 5 
 jobs 
 83 
 3,100 
 8.63 
 274.52 
 
 
 6 
 sector 
 50 
 866 
 8.47 
 239.69 
 
 
 7 
 economy 
 70 
 3,758 
 8.25 
 865.99 
 
 
 8 
 growth 
 51 
 1,762 
 8.24 
 176.16 
 
 
 9 
 segment 
 34 
 235 
 8.12 
 227.17 
 
 
 10 
 vital 
 45 
 1,707 
 8.08 
 147.53 
 
 
 11 
 american 
 165 
 18,173 
 7.95 
 235.41 
 
 
 12 
 competitive 
 34 
 842 
 7.92 
 139.11 
 
 
 13 
 nations 
 82 
 7,412 
 7.91 
 142.89 
 
 
 14 
 growing 
 38 
 1,432 
 7.91 
 125.05 
 
 
 15 
 sectors 
 28 
 131 
 7.88 
 210.88 
 
 
 16 
 help 
 76 
 6,905 
 7.87 
 131.80 
 
 
 17 
 entire 
 47 
 3,482 
 7.73 
 97.56 
 
 
 18 
 agricultural 
 41 
 2,598 
 7.72 
 96.20 
 
 
 19 
 impact 
 36 
 1,878 
 7.71 
 96.93 
 
 
 20 
 domestic 
 38 
 2,193 
 7.71 
 95.44 
 
 
 
 
 
 Report based on word tokens 
 


 
 Context tokens left: 5, context tokens right: 5 
 


 
 Filtered tokens by minimum collocation frequency (5) 
 


 
 Keywords filtered based on p-value 0.0001 with Bonferroni correction (based on 864 tests) 
 


 
 Unique collocates: 123 
 


 
 Showing 20 rows 
 


 
 Page 1 of 7

CPU times: user 1.83 s, sys: 947 ms, total: 2.77 s
Wall time: 2.59 s


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()